## Computer Assignment #1 <br><br>
### mohammadreza mohammadhashemi 810100206 <br><br>
### Genetic Algorithm(Finding coefficients of polynomials)<br><br>
#### project goal : <br><br>
The purpose of this project is to practice and learn the use of genetic algorithms to solve problems with a large state space, which is done for finding polynomial coefficients. <br><br>
#### Problem Description :<br><br>
We are given the coordinates of a number of points and the degree of the desired polynomial for curve fitting.We have to find the polynomial coefficients using genetic algorithms for the given polynomial degree that performs the best fit.<br>
It is possible that the real polynomial coefficients are non-integer numbers that due to our limitation to integer coefficients, we cannot find exactly the original polynomial coefficients. Our task is to provide the best estimate.<br><br>
In the following section, a test case is automatically generated for the problem and stored in a csv file so that the main code of the project uses this test case.<br><br>
The variables of this section are:<br><br>
polynomial_degree : This variable specifies the polynomial degree of the test case, which is used both in the test case generation function and as an input in the main code to find the coefficients.<br><br>
x_lower_bound : This variable specifies the lowest value of the x coordinate of the points which can generated in testcase.<br><br>
x_upper_bound : This variable specifies the highest value of the x coordinate of the points which can generated in testcase.<br><br>
c_lower_bound : This variable specifies the lowest value of the coefficients which we can generated in testcase.<br><br>
c_upper_bound : This variable specifies the highest value of the coefficients which we generated in testcase.<br><br>
n : This variable specifies the number of points that generated in the testcase generator.<br><br>
real_answer : This variable stores the coefficients generated in the test case to compare with the coefficient that the program finds.<br>
we need to reverse this variable becuas of the difference bitween the polyval function and the way we used to calculate the result of the polynomial in the testcase generator function.<br><br>


In [1]:
import csv
import random

polynomial_degree = 3
x_lower_bound = -15
x_upper_bound = 15
c_lower_bound = 0
c_upper_bound = 5
n = 4
real_answer = []

def generate_polynomial_data(degree, x_lower_bound, x_upper_bound, n , c_lower_bound , c_upper_bound , real_answer):

  for i in range(0 , degree+1):
    real_answer.append(random.randint(c_lower_bound , c_upper_bound))

  x_values = set()
  while len(x_values) < n:
    x_value = random.randint(x_lower_bound, x_upper_bound)
    x_values.add(x_value)

  y_values = []
  for x in x_values:
    y_value = 0
    for i in range(degree + 1):
      y_value += real_answer[i] * x ** i
    y_values.append(y_value)

  return list(zip(x_values, y_values))

def create_testcase(data, file_path):

  with open(file_path, "w", newline="") as csv_file:
    writer = csv.writer(csv_file)
    writer.writerows([])

  with open(file_path, "w", newline="") as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(["x", "y"])

  with open(file_path, "a", newline="") as csv_file:
    writer = csv.writer(csv_file)
    writer.writerows(data)

data = generate_polynomial_data(polynomial_degree, x_lower_bound, x_upper_bound, n , c_lower_bound , c_upper_bound , real_answer)
real_answer.reverse()
file_path = "gfd.csv"
create_testcase(data, file_path)

In the next section, we will load the input points coordinates for the curve fitting process from the CSV file.<br><br>

In [2]:
import pandas as pd
data =  pd.read_csv("points.csv")
x = data["x"]
y = data["y"]

To solve this problem with genetic algorithms, we have polinomial degree + 1 long chromosomes, which in the following section, each gene of each chromosome is randomly generated in the range specified in the input to generate the initial random population.<br><br>

In [3]:
import random

def create_first_gen(population_size, chromosome_length, lower_limit, upper_limit):

  population = []
  for i in range(population_size):
    chromosome = []
    for j in range(chromosome_length):
      chromosome.append(random.randint(lower_limit, upper_limit))
    population.append(chromosome)
  return population

In the following section, the population fitness calculation function is given.<br>
That the sum of the absolute value of the distance of the calculated ys by the coefficients of each chromosome with the actual value of each point is used as a fitness function.<br><br>
Also, after calculating the fitness of each chromosome, in addition to the value of the fitness of that chromosome, we also store the chromosome index in the tuple related to that chromosome, which we will see its use in the next sections.<br><br>

In [4]:
import numpy as np

class chromosome_fit :
  fitness_value = -1
  chromosome_index = -1
  def __init__(self, fit, index):
    self.fitness_value = fit
    self.chromosome_index = index

def fitness(population, points):
  fitnesses = []
  for chromosome in population:    
    sum_squared_differences = 0.0
    for x, y in points:
      y_predicted = np.polyval(chromosome, x)
      sum_squared_differences += abs(y - y_predicted)
    fitnesses.append(chromosome_fit(sum_squared_differences ,len(fitnesses)))
  return fitnesses

In the following section, we select parents from the existing population in order to use these parents to build the next generation population. The probability of each chromosome of this generation being selected as a parent is equal to the fitness rating of this chromosome divided by the sum of all fitness ratings.<br><br>
In the rand_index function, a random number is first generated, and then, according to the indexes guide, in each element contains the sum of k = 1 to index , the index of the parent is returned based on this random number and ranks guide to meet the possibilities.<br><br>
Also, the first 10% of the population will always be parents<br><br>

In [5]:
def rand_index(sum_of_ranks , rank_guide , population_size) :
    random_number = random.randint(1, int(sum_of_ranks))
    for i in range(0 , population_size) :
        if random_number < rank_guide[i] :
            return i-1
        elif random_number == rank_guide[i] :
            return i

def select_parrents(fitnesses , population , rank_guide) :
    parents = []
    sum_of_ranks = (len(population) * (len(population)+1))/2
    for i in range(len(population)-int(len(population)/10) , len(population)):
        parents.append((population[fitnesses[i].chromosome_index] , i)) 
    for i in range (0 , len(population) - int(len(population)/10)) :
        fit_index = rand_index(sum_of_ranks , rank_guide , len(population))
        parents.append((population[fitnesses[fit_index].chromosome_index] , fit_index))
    return parents

In the following section, the crossover phase هs implemented<br><br>
The process started from the cross-phase function. First, the first ten percent of the parents' population, who themselves caried, and in fact they are the first 10 percent of the population of the last generation with the lowest value in the fitness function, are directly caried to the next generation in carry function.<br><br>

Then, the crossover process is performed in the handle_cross_over function, which is performed by calling the find_children function. In this function, the repeated_flag is set, considering that the process of selecting parents is a random process, and there is a possibility of elements being duplicated in parents. so this flag checks whether this chromosome already exists in the next generation or not. If it exists, this pair of chromosomes will definitely crossed over, but if they did not exist before, they have the probeblity of p_cross_over to get crossed over and 1-p_cross_over to get carried.<br><br>

The P_cross_over variable is defined in this section, which can be used to change the probability of the crossover process.<br><br>

I used uniform crossover method and the probability of having cut points is not static and based on the fitnesses of those chromosomes.<br><br>

In [6]:
def carry (parents , next_gen) :
    carrys_count = int(len(parents)/10) 
    for i in range(0 , carrys_count) :
        next_gen.append(parents[0][0])
        del parents[0]

def find_children (parents , first_child , second_child , i , next_gen) :
    p_cross_over = 0.7
    repeat_flag = False
    
    if parents[2*i][0] in next_gen :
        repeat_flag = True
    elif parents[2*i+1][0] in next_gen :
        repeat_flag = True    
    
    if (random.uniform(0 , 1) > p_cross_over and not repeat_flag) :
        for j in range(0,len(parents[2*i][0])) :
            first_child.append(parents[2*i][0][j])
            second_child.append(parents[2*i+1][0][j])
    else :
        cross_patern = [1]
        first_child.append(parents[2*i][0][0])
        second_child.append(parents[2*i+1][0][0])
        p_cross_point = (min(parents[2*i][1] , parents[2*i+1][1])/(parents[2*i][1] + parents[2*i+1][1]))
        for j in range(1,len(parents[2*i][0])) :
            if random.uniform(0,1) > p_cross_point :
                cross_patern.append(cross_patern[j-1])
            else :
                cross_patern.append((cross_patern[j-1]+1) % 2)
            first_child.append(parents[2*i + ((cross_patern[j] + 1) % 2)][0][j])
            second_child.append(parents[2*i + ((cross_patern[j]) % 2)][0][j])
    
def handle_cross_over (parents , next_gen , i) : 
    first_child = []
    second_child = []
    mode = find_children(parents , first_child , second_child , i , next_gen)
    next_gen.append(first_child)
    next_gen.append(second_child)

def cross_phase(parents) :
    next_gen = []
    carry(parents , next_gen)
    random.shuffle(parents)
    for i in range(0 , int(len(parents)/2 )) :
        handle_cross_over(parents , next_gen , i)
        
    return next_gen
    

In the following section, the phase of mutation is implemented, which occurs for each gene of each chromosome by the probability of p_mutation_multiplied*(0.9995 to the power of the generation number of the mutation process).

In [7]:
import random

def mutate(gen , cycle , higher_bound , lower_bound) :
    p_mutation = 0.3*(0.9995**cycle)
    for i in range(0 , len(gen)) :
        if random.uniform(0,1) < p_mutation :
            gen[i] = random.randint(lower_bound , higher_bound)

def mutation_phase(population, cycle , higher_bound , lower_bound) :
    for x in population :
        mutate(x , cycle , higher_bound , lower_bound)

And finally, in the following section, the mine function is implemented, which first produces an initial population, then performs the process of producing a new generation for the specified number of generations, and stores the best answer in the whole process, and if it ever finds the original answer to the problem that has the fitness = 0 breaks the loop and print the answer.<br><br>
Variable cooficient_lower_bound specified the lower limit of the coeficients assumed the answer will have so the program will not generate any coeficient in any gen lower than that value. <br><br>
Variable cooficient_higer_bound specified the highest limit of the coeficients assumed the answer will have so the program will not generate any coeficient in any gen higher than that value. <br><br>
Variable generations_count specified how many cycles will program generates new generations in the worst case.<br><br>
Variable population_size specified the size of population in all program prosess.<br><br>

In [8]:
import sys
import copy

polynomial_degree = 2
cooficient_lower_bound = -5
cooficient_higher_bound = 5
generations_count = 4000
population_size = 150
pop = create_first_gen(population_size , polynomial_degree+1 , cooficient_lower_bound , cooficient_higher_bound)
rank_guide = []
min_fit = float("inf")
answer = [];

for i in range(0 , len(pop)) :
    rank_guide.append(((i+1)*(i+2))/2)


for i in range(0 , generations_count) :
    fit = fitness(pop , tuple(zip(x , y)))
    temp_min = min(fit , key = lambda x:x.fitness_value)
    if (temp_min.fitness_value < min_fit) :
        min_fit = temp_min.fitness_value
        answer = copy.deepcopy(pop[temp_min.chromosome_index])
    
    if min_fit == 0 :
        break
    
    fit.sort(reverse = True , key = lambda x:x.fitness_value)
    parents = select_parrents(fit , pop , rank_guide)
    pop = copy.deepcopy(cross_phase(parents))
    mutation_phase(pop , i  , cooficient_higher_bound, cooficient_lower_bound)

print ("real answer was : " + str(real_answer) + "\nand found answer is : ")
print (answer)



real answer was : [2, 0, 1, 5]
and found answer is : 
[1, 3, 1]


## QUESTIONS : <br><br>
### QUESTION 1 : <br><br>
A very small population size causes the algorithm to get stuck in the local optimum and not find the real solution. Also, if this problem is solved, it will suffer from computational overhead.<br>
A very large initial population causes an irrational increase in the execution time of the algorithm and the space cost of the algorithm.<br><br>
### QUESTION 2 : <br><br>
It increases the accuracy due to the increase in diversity in the chromosomes, which increases the chance of finding a suitable answer to the problem, but due to the increase in the amount of memory cost and also due to the increase in the length of the iterations, it increases the execution time.
### QUESTION 3 : <br><br>
The crossover operation combines the existing solutions, and since in the selection phase, the probability of choosing better members of the population of the previous generation is higher, then by combining the best solutions of the previous generation, on average, in the next generation, we generated solutions that are closer to the solution of the problem.<br>
The mutation operation produces new answers outside the population of previous generations and increases the diversity of answers in the population, which increases the probability of reaching the right answer to the problem!<br>
But it is not possible to use one of these processes alone, because if we only use crossover, we will be limited to the combinations of our first generation and we will not have any randomness in next generations to increase the diversity of answers, and this causes that if the initial population dont cover the answer, the algorithm cannot reach the answer.<br>
Also, if we only use the mutation operation, since all the changes are random, there is no direction towards better answers in the population, and the answers produced move in all directions at the same speed on average.<br><br>
### QUESTION 4 : <br><br>
Running some optimizations to find the efficient values of the mutation and crossover p values can make the program more efficient . Also, using fixed size arrays and optimizing the functions can improve the performance .<br><br>
### QUESTION 5 : <br><br>
Due to the propagation of duplicate elements in next generations, it is possible that after a few generations, there will not be so much change in the production of new generations , which can be solved by checking whether duplicate answers are propagated to the next generation, which is done in the code. and by increasing the mutation probablity of the mutation we can increase the divesity .<br><br>

### QUESTION 6 : <br><br>
Specifying the maximum number of program cycles.<br>
Also, if necessary, it is possible to monitor the changes in the variance of the fitness of the generations, and if these changes are not so great for a significant number of generations, the execution of the program can be terminated and the closest solution found can be provided.<br><br>

### QUESTION 7 : <br><br>
It only affects the calculation time of the fitness function and this increase could have been a linear relationship with the increase in the polynomial degree but considering the time overhead to calculate higer powers make me realize this over head can be o(n^2) and omega(n) based on how efficient is the power operator .<br><br>

### QUESTION 8 : <br><br>
Due to the repeated use of the input coordinates for each chromosome to calculate the fitness in each cycle of the program, it increases the program execution time. Also, if the degree of polynomial and the number of points, especially the number of points, exceed a certain value, we will experience an overflow during the calculation of fitness.<br><br>
